In [54]:
import numpy as np

### Decision Tree Classifier Task

We are provided with a toy dataset that classifies drinks into three categories: **Wine**, **Beer**, and **Whiskey**. The classification is based on three features:

1. **Alcohol Content (%)**
2. **Sugar Content (g/L)**
3. **Color** (0: light, 1: dark)

Our goal is to build a **Decision Tree Classifier** that predicts the type of drink given its features.

#### Toy Dataset:
| Alcohol Content (%) | Sugar (g/L) | Color (0: light, 1: dark) | Drink Type |
|----------------------|-------------|---------------------------|------------|
| 12.0                | 1.5         | 1                         | Wine       |
| 5.0                 | 2.0         | 0                         | Beer       |
| 40.0                | 0.0         | 1                         | Whiskey    |
| 13.5                | 1.2         | 1                         | Wine       |
| 4.5                 | 1.8         | 0                         | Beer       |
| 38.0                | 0.1         | 1                         | Whiskey    |
| 11.5                | 1.7         | 1                         | Wine       |
| 5.5                 | 2.3         | 0                         | Beer       |

In [55]:
# Sample dataset
data = [
    [12.0, 1.5, 1, 'Wine'],
    [5.0, 2.0, 0, 'Beer'],
    [40.0, 0.0, 1, 'Whiskey'],
    [13.5, 1.2, 1, 'Wine'],
    [4.5, 1.8, 0, 'Beer'],
    [38.0, 0.1, 1, 'Whiskey'],
    [11.5, 1.7, 1, 'Wine'],
    [5.5, 2.3, 0, 'Beer']
]

# Map string labels to integers for processing
label_map = {'Wine': 0, 'Beer': 1, 'Whiskey': 2}

# Replace label strings with corresponding integers
for row in data:
    row[3] = label_map[row[3]]

# Convert list of lists to NumPy array with float values
data = np.array(data, dtype=float)

In [56]:
# Split features (X) and labels (y)
X = data[:, :-1]            # All columns except the last one as features
y = data[:, -1].astype(int) # Last column as labels, converted to integers

### Gini Impurity Calculation

This function calculates the **Gini impurity** for a set of class labels, which measures node "purity" in decision trees.

#### Formula:
$$
Gini = 1 - \sum_{i=1}^{n} (p_i)^2
$$
Where \( p_i \) is the probability of class \( i \) in the labels.

#### Steps:
1. **Count Classes**: `np.unique(labels, return_counts=True)` gets frequency counts for each class
2. **Calculate Probabilities**: Convert counts to probabilities
3. **Compute Impurity**: Subtract sum of squared probabilities from 1

In [57]:
def gini_impurity(labels):
    _, counts = np.unique(labels, return_counts=True)  # Get counts of each class (ignore the actual label values with '_')
    probabilities = counts / counts.sum()  # Class probabilities
    gini = 1 - np.sum(probabilities ** 2)  # Gini impurity formula
    return gini

### Finding the Best Split in a Decision Tree

This function identifies the optimal feature and threshold to split the data, minimizing the weighted Gini impurity.

#### Key Formula (Weighted Gini):
$$
\text{Weighted Gini} = \left(\frac{n_{\text{left}}}{n_{\text{total}}}\right)Gini_{\text{left}} + \left(\frac{n_{\text{right}}}{n_{\text{total}}}\right)Gini_{\text{right}}
$$

#### How It Works:
1. **Initialization**:  
   - Starts with infinite `best_gini` (seeking lower values)
   - Tracks best feature index and threshold

2. **Feature Iteration**:  
   - Examines each feature (Alcohol%, Sugar, Color)
   - Generates candidate thresholds between sorted unique values

3. **Threshold Evaluation**:  
   - Splits data using `feature ≤ threshold`
   - Skips invalid splits (empty left/right subsets)
   - Computes weighted Gini impurity for valid splits

4. **Optimal Selection**:  
   - Retains the split with lowest weighted Gini
   - Returns best feature index, threshold, and impurity score

#### Key Characteristics:
- Exhaustively checks all possible splits
- Uses midpoints between consecutive values as thresholds
- Prefers splits that create the purest child nodes
- Runtime complexity: O(features × thresholds × samples)

In [58]:
def best_split(X, y):
    n_samples, n_features = X.shape  # Get number of samples and features
    best_gini = float('inf')         # Initialize best Gini as infinity (we want to minimize it)
    best_feature = None              # Track the best feature to split on
    best_threshold = None            # Track the best threshold for the split

    current_gini = gini_impurity(y)  # Gini impurity of the current node (before splitting)
    
    for feature_idx in range(n_features):
        # Get sorted unique values for this feature
        feature_values = np.sort(np.unique(X[:, feature_idx]))
        
        # Compute midpoints between consecutive unique values as candidate thresholds
        if len(feature_values) > 1:
            thresholds = [(feature_values[i] + feature_values[i+1]) / 2 
                          for i in range(len(feature_values) - 1)]
        else:
            thresholds = [feature_values[0]]  # Only one unique value — use it as threshold
            
        for threshold in thresholds:
            # Split data into left and right based on threshold
            left_mask = X[:, feature_idx] <= threshold
            right_mask = ~left_mask  # Everything else goes to the right
            
            # Skip split if either side is empty
            if np.sum(left_mask) == 0 or np.sum(right_mask) == 0:
                continue
                
            y_left, y_right = y[left_mask], y[right_mask]
            
            # Calculate Gini impurity for both sides
            gini_left = gini_impurity(y_left)
            gini_right = gini_impurity(y_right)
            
            # Compute weighted average of the two impurities
            n_left, n_right = len(y_left), len(y_right)
            weighted_gini = (n_left * gini_left + n_right * gini_right) / n_samples
            
            # Keep track of the best (lowest impurity) split
            if weighted_gini < best_gini:
                best_gini = weighted_gini
                best_feature = feature_idx
                best_threshold = threshold

    return best_feature, best_threshold, best_gini  # Return the best split found

In [59]:
class Node:
    def __init__(self, feature_index=None, threshold=None, left=None, right=None, value=None):
        self.feature_index = feature_index  # Index of feature used for splitting
        self.threshold = threshold          # Threshold value for the split
        self.left = left                    # Left child node (subtree)
        self.right = right                  # Right child node (subtree)
        self.value = value                  # Class label if it's a leaf node

    def is_leaf_node(self):
        return self.value is not None  # Returns True if node is a leaf (i.e., has a class label)

In [60]:
def majority_label(y):
    values, counts = np.unique(y, return_counts=True)  # Get unique labels and their counts
    return values[np.argmax(counts)]  # Return the label with the highest count (majority class)

### Decision Tree Construction Process

This recursive function builds a decision tree by splitting nodes until stopping criteria are met.

#### Node Types:
- **Decision Node**:  
  Contains:
  - `feature_index`: Feature used for splitting (e.g., Alcohol%)
  - `threshold`: Split boundary (e.g., ≤12.5%)
  - `left/right`: Child nodes
- **Leaf Node**:  
  Contains `value`: Final class prediction (Wine/Beer/Whiskey)

#### Workflow:
1. **Check Stopping Conditions**:
   - All samples same class → Create leaf
   - Reached depth limit → Create majority-class leaf
   - Insufficient samples → Create majority-class leaf

2. **Find Optimal Split**:
   - Uses `best_split()` to minimize weighted Gini impurity

3. **Create Subtrees**:
   - Left branch: Samples where `feature ≤ threshold`
   - Right branch: Remaining samples
   - Recursively repeat process for each branch

#### Control Parameters:
| Parameter | Purpose |
|-----------|---------|
| `max_depth=5` | Prevents overcomplex trees |
| `min_samples=2` | Avoids splits on tiny groups |

In [61]:
def build_tree(X, y, depth=0, max_depth=5, min_samples=2):
    n_samples, n_features = X.shape  # Get number of samples and features
    
    # Stop if all labels are the same (pure node)
    if len(np.unique(y)) == 1:
        return Node(value=y[0])  # Make leaf with the common label
    
    # Stop if maximum depth reached or not enough samples to split
    if depth >= max_depth or n_samples < min_samples:
        return Node(value=majority_label(y))  # Make leaf with majority label
    
    # Find the best feature and threshold to split on
    feature_idx, threshold, _ = best_split(X, y)
    
    # If no valid split was found (e.g. constant features)
    if feature_idx is None:
        return Node(value=majority_label(y))  # Make leaf with majority label
    
    # Split dataset into left and right branches
    left_mask = X[:, feature_idx] <= threshold
    right_mask = ~left_mask
    
    # Safety check in case split is invalid (shouldn't usually happen)
    if np.sum(left_mask) == 0 or np.sum(right_mask) == 0:
        return Node(value=majority_label(y))  # Make leaf with majority label
    
    # Recursively build left and right subtrees
    left_subtree = build_tree(X[left_mask], y[left_mask], depth + 1, max_depth, min_samples)
    right_subtree = build_tree(X[right_mask], y[right_mask], depth + 1, max_depth, min_samples)
    
    # Return decision node with split info and subtrees
    return Node(feature_index=feature_idx, threshold=threshold, 
                left=left_subtree, right=right_subtree)

In [62]:
def predict_sample(node, x):
    # If current node is a leaf, return its value (class label)
    if node.is_leaf_node():
        return node.value

    # Recursively follow the left or right subtree based on threshold
    if x[node.feature_index] <= node.threshold:
        return predict_sample(node.left, x)
    else:
        return predict_sample(node.right, x)

In [63]:
def predict(tree, X):
    # Apply the predict_sample function to each sample in X and return the predicted labels
    return np.array([predict_sample(tree, x) for x in X])

In [64]:
tree = build_tree(X, y)

In [65]:
test_data = np.array([
    [6.0, 2.1, 0],   # Expected: Beer
    [39.0, 0.05, 1], # Expected: Whiskey
    [13.0, 1.3, 1]   # Expected: Wine
])

### Decision Tree Prediction Process

This code uses the trained decision tree to classify test data into drink types:

1. **Tree Traversal**:  
   For each test sample, the tree is traversed from root to leaf:
   - Decision nodes check features against thresholds (e.g., "Alcohol ≤ 40.0?")
   - Follow left/right branches until reaching a leaf node

2. **Label Conversion**:  
   Converts numerical predictions to drink names using:
   `types = {0: "Wine", 1: "Beer", 2: "Whiskey"}`

In [66]:
# Predict the labels for the test data
predicted_labels = predict(tree, test_data)

# Map numeric labels back to the original drink types
types = {0: "Wine", 1: "Beer", 2: "Whiskey"}
predicted_types = [types[label] for label in predicted_labels]

# Print the predicted drink types
print("Predicted Drink Types:", predicted_types)

Predicted Drink Types: ['Beer', 'Whiskey', 'Wine']
